In [1]:
from src.dataset import GEM1Dataset
from torch_geometric.data import DataLoader

train_data = GEM1Dataset(mode="train")
test_data = GEM1Dataset(mode="test")

train_loader = DataLoader(train_data, batch_size=32, shuffle=True, drop_last=True, follow_batch=["edge_attr"])
test_loader = DataLoader(test_data, batch_size=32, shuffle=False, drop_last=False, follow_batch=["edge_attr"])
batch = next(iter(test_loader))
batch

Processing...


457it [00:08, 53.99it/s]


Done!


Batch(batch=[1915], bond_bond_angles_ex=[11114], bond_bond_angles_g=[11114], bond_bond_index=[2, 11114], bond_lengths_ex=[4090], bond_lengths_g=[4090], edge_attr=[4090, 3], edge_attr_batch=[4090], edge_index=[2, 4090], pos_ex=[1915, 3], pos_g=[1915, 3], ptr=[33], x=[1915, 9])

In [3]:
import pandas as pd


sub_df = pd.read_csv("data/sample_submission.csv")
sub_df

,index,Reorg_g,Reorg_ex
0,test_0,0,0
1,test_1,0,0
2,test_2,0,0
3,test_3,0,0
4,test_4,0,0
...,...,...,...
452,test_452,0,0
453,test_453,0,0
454,test_454,0,0
455,test_455,0,0


In [4]:
import torch
from tqdm import tqdm
import numpy as np
from src.model import GEM1Model
import yaml
from easydict import EasyDict

with open("configs/model1.yaml", "r") as f:
    cfg = EasyDict(yaml.safe_load(f))


preds = []

device = torch.device("cuda:1")

model = GEM1Model(**cfg.model).to(device)
model.load_state_dict(torch.load("outputs/model1/ckpts/best.pt"))

for batch in tqdm(test_loader):
    batch = batch.to(device)
    pred = model(batch).detach().cpu().numpy()
    preds.append(pred)

preds = np.concatenate(preds)
preds.shape

100%|██████████| 15/15 [00:07<00:00,  2.08it/s]


(457, 2)

In [6]:
sub_df[sub_df.columns[1:]] = preds
sub_df.to_csv("outputs/model1/submissions/submission.csv", index=False)